In [108]:
# Import libraries

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,learning_curve, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

from sklearn.linear_model import LassoCV,ElasticNetCV,Lasso,ElasticNet
from sklearn.kernel_ridge import KernelRidge


# 아래 추가적인 패키지가 설치가 되어 있어야 함!!!!!!!
from xgboost import XGBRegressor
from mlxtend.regressor import StackingRegressor


# 시간
import time

#### 정규화 작업

In [109]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
#X_train = RobustScaler().fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns = df.columns )

X_test = scaler.transform(X_test)
X_test = pd.DataFrame(X_test, columns = df.columns)
#X_test = RobustScaler().fit_transform(X_test)

In [110]:
y_df.head()

0    12.247699
1    12.109016
2    12.317171
3    11.849405
4    12.429220
Name: SalePrice, dtype: float64

In [111]:
y_df.shape

(1456,)

In [112]:
#  참고) 위에서 제일 원본 데이터는 208500이였는데, 이것을 log1p로 변환한 것을 정답지로 놓고 하는 것이니
# 하기 전에 맞게 변환이 되었는지 확인!!!!!!
np.log1p(208500)

12.24769911637256

#### 모델의 종류들 : 평가는 root-mean-squared-error 로 평가

        Ridge Regression
        Lasso Regression
        ElasticNet Regression
        Support Vector Machine
        Random Forest
        XG Boost

##### 평가 및 kfold 사용 준비

In [113]:
kfold = KFold(n_splits=6, random_state= 0, shuffle = True)

In [114]:
def rmsle_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y_df, 
                                   scoring="neg_mean_squared_error", cv = kfold))
    return(rmse)

##### Random Forest

In [ ]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,learning_curve, cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

* ref) https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

In [115]:
RFC = RandomForestRegressor(random_state = 1234, n_jobs=-1)

rf_param_grid = {
    "n_estimators" : [100],
    
    # The function to measure the quality of a split  : 기본 mse - mean squared error
    # "criterion":['mse', 'mae'],
    "criterion":['mse'],
    
    "max_depth" : [None ],
    
    # Default : 2
    # "min_samples_split":[1, 2],
    "min_samples_leaf" : [2],
    
    # 
    # "min_weight_fraction_leaf" : [],
    
    # The number of features to consider when looking for the best split:
    # If “auto”, then max_features=n_features.
    # If “sqrt”, then max_features=sqrt(n_features).
    # If “log2”, then max_features=log2(n_features).
    # If None, then max_features=n_features.
    
    # "max_features" : ["auto", "sqrt"],
    "max_features" : ["auto"],
    
    # Grow trees with max_leaf_nodes in best-first fashion.
    "max_leaf_nodes" : [None],
    
    # A node will be split if this split induces a decrease of the impurity greater than or equal to this value.
    # "min_impurity_decrease" : 
    
    # Threshold for early stopping in tree growth. A node will split if its impurity is above the threshold, otherwise it is a leaf.
    # "min_impurity_split" : []
    
    # Whether bootstrap samples are used when building trees.: Default - True
    "bootstrap" : [True],
    
    # whether to use out-of-bag samples to estimate the R^2 on unseen data.
    # Default : False
    "oob_score" : [True, False],
    
    "n_jobs" : [-1],
    
    # When set to True, reuse the solution of the previous call to fit and add more estimators to the ensemble, otherwise, just fit a whole new forest.
    "warm_start" : [True, False]
}


gsRFC = GridSearchCV(RFC,
                     param_grid = rf_param_grid, 
                     cv = kfold, 
                     scoring = "neg_mean_squared_error", 
                     n_jobs = -1, verbose = 1)
gsRFC.fit(X_train,y_df)
gsRFC_best = gsRFC.best_estimator_
print(gsRFC_best)

Fitting 6 folds for each of 4 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   28.2s finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=True,
                      random_state=1234, verbose=0, warm_start=True)


In [116]:
print("Random Forest mean score:", rmsle_cv(gsRFC_best).mean())
print("Random Forest std:", rmsle_cv(gsRFC_best).std())

Random Forest mean score: 0.1376128355606156
Random Forest std: 0.010441275582848748


In [117]:
y_submission_5 = np.expm1(gsRFC_best.predict(X_test))

In [121]:
gsRFC_best.predict(X_test)

array([11.71734382, 11.96353297, 12.06428059, ..., 11.95490371,
       11.63400122, 12.32743214])

In [122]:
y_submission_5

array([122680.12925131, 156925.52366703, 173559.33903746, ...,
       155577.18900323, 112870.04188337, 225805.14351727])